# Data treatment
Nessa seção, iremos filtrar os dados do arquivo CSV que geramos anteriormente em apenas os CIDs correspondentes a suicídio.


In [1]:
# Imports
import csv
import json
import pandas as pd

In [38]:
# Dataset com os dados de suicídio de 2011 a 2015
df_suicidio = pd.read_csv('data/suicidios.csv', sep=';', low_memory=False)

In [39]:
df_suicidio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55639 entries, 0 to 55638
Data columns (total 23 columns):
DTOBITO       55639 non-null int64
HORAOBITO     41549 non-null object
DTNASC        55327 non-null float64
IDADE         55639 non-null int64
SEXO          55639 non-null int64
RACACOR       53574 non-null float64
ESTCIV        53598 non-null float64
OCUP          41825 non-null float64
CODMUNRES     55639 non-null int64
LOCOCOR       55639 non-null int64
CODMUNOCOR    55639 non-null int64
IDADEMAE      8 non-null float64
ESCMAE        8 non-null float64
OCUPMAE       7 non-null float64
QTDFILVIVO    7 non-null float64
QTDFILMORT    6 non-null float64
LINHAA        53354 non-null object
LINHAB        37960 non-null object
LINHAC        10436 non-null object
LINHAD        4038 non-null object
LINHAII       4520 non-null object
ESC2010       39926 non-null float64
ESCMAE2010    6 non-null float64
dtypes: float64(11), int64(6), object(6)
memory usage: 9.8+ MB


In [40]:
# Visto que temos algumas colunas não preenchidas, iremos excluir elas
df_suicidio = df_suicidio.drop(columns=['IDADEMAE', 'ESCMAE', 'OCUPMAE',
                                        'QTDFILVIVO', 'QTDFILMORT'])

In [72]:
estado_civil = {
    1: 'Solteiro',
    2: 'Casado',
    3: 'Viúvo',
    4: 'Separado',
    5: 'União',
    9: 'Ignorado'
}

sexo = {
    0: 'Não informado',
    1: 'Masculino',
    2: 'Feminino',
    9: 'Ignorado'
}

raca_cor = {
    1: 'Branca',
    2: 'Preta',
    3: 'Amarela',
    4: 'Parda',
    5: 'Indígena'
}

cores = {
    'M': ['#A9A9A9', '#000000', '#FF7F50', '#FFD700', '#FF8C00'],
    'F': ['#C0C0C0', '#696969', '#FF6347', '#F0E68C', '#FFA500']
}

In [44]:
df_suicidio['ESTCIV_LABEL'] = df_suicidio['ESTCIV'].apply(lambda x: (estado_civil[x] if x in estado_civil.keys() else x))
df_suicidio['SEXO_LABEL'] = df_suicidio['SEXO'].apply(lambda x: (sexo[x] if x in sexo.keys() else x))
df_suicidio['RACACOR_LABEL'] = df_suicidio['RACACOR'].apply(lambda x: (raca_cor[x] if x in raca_cor.keys() else x))

In [92]:
dict_chart1 = list()
paddings = [0.4, 0.45]
points = [-0.3, -0.15, 0.0, 0.15, 0.3]
count = 0

for sex in sexo.values():
    for racacor in raca_cor.values():
        if sex[0] in ['N', 'I']:
            continue
        
        dict_data = dict()
        dict_data["name"] = "({}) {}".format(sex[0], racacor)
        list_values = list()
        
        for estciv in estado_civil.values():
            if estciv == 'Ignorado' :
                continue
                
            list_values.append(df_suicidio[(df_suicidio.ESTCIV_LABEL == estciv)
                                           & (df_suicidio.SEXO_LABEL == sex)
                                           & (df_suicidio.RACACOR_LABEL == racacor)]['DTOBITO'].count())
        
        dict_data["data"] = list_values
        dict_data["pointPadding"] = paddings[0 if sex[0] == 'M' else 1]
        dict_data["pointPlacement"] = points[count]
        dict_data["color"] = cores[sex[0]][count]
        dict_chart1.append(dict_data)
        
        count += 1
    count = 0

dict_chart1

[{'name': '(M) Branca',
  'data': [9396, 6728, 824, 1773, 741],
  'pointPadding': 0.4,
  'pointPlacement': -0.3,
  'color': '#A9A9A9'},
 {'name': '(M) Preta',
  'data': [1511, 492, 68, 84, 143],
  'pointPadding': 0.4,
  'pointPlacement': -0.15,
  'color': '#000000'},
 {'name': '(M) Amarela',
  'data': [62, 61, 16, 10, 7],
  'pointPadding': 0.4,
  'pointPlacement': 0.0,
  'color': '#FF7F50'},
 {'name': '(M) Parda',
  'data': [10223, 3928, 386, 719, 1063],
  'pointPadding': 0.4,
  'pointPlacement': 0.15,
  'color': '#FFD700'},
 {'name': '(M) Indígena',
  'data': [280, 35, 3, 2, 48],
  'pointPadding': 0.4,
  'pointPlacement': 0.3,
  'color': '#FF8C00'},
 {'name': '(F) Branca',
  'data': [2470, 2098, 588, 762, 244],
  'pointPadding': 0.45,
  'pointPlacement': -0.3,
  'color': '#C0C0C0'},
 {'name': '(F) Preta',
  'data': [327, 123, 52, 28, 24],
  'pointPadding': 0.45,
  'pointPlacement': -0.15,
  'color': '#696969'},
 {'name': '(F) Amarela',
  'data': [26, 16, 17, 1, 3],
  'pointPadding': 0

In [12]:
df_chart1.to_json('data/chart1.json')

Códigos do IBGE para cada Unidade Federal (UF):

| Região       | Código | Unidade da Federação | UF |
| ------------ | ------ | -------------------- | -- |
| Norte        | 11     | Rondônia             | RO |
|              | 12     | Acre                 | AC |
|              | 13     | Amazonas             | AM |
|              | 14     | Roraima              | RR |
|              | 15     | Pará                 | PA |
|              | 16     | Amapá                | AP |
|              | 17     | Tocantins            | TO |
| Nordeste     | 21     | Maranhão             | MA |
|              | 22     | Piauí                | PI |
|              | 23     | Ceará                | CE |
|              | 24     | Rio Grande do Norte  | RN |
|              | 25     | Paraíba              | PB |
|              | 26     | Pernambuco           | PE |
|              | 27     | Alagoas              | AL |
|              | 28     | Sergipe              | SE |
|              | 29     | Bahia                | BA |
| Sudeste      | 31     | Minas Gerais         | MG |
|              | 32     | Espírito Santo       | ES |
|              | 33     | Rio de Janeiro       | RJ |
|              | 35     | São Paulo            | SP |
| Sul          | 41     | Paraná               | PR |
|              | 42     | Santa Catarina       | SC |
|              | 43     | Rio Grande do Sul    | RS |
| Centro-Oeste | 50     | Mato Grosso do Sul   | MS |
|              | 51     | Mato Grosso          | MT |
|              | 52     | Goiás                | GO |
|              | 53     | Distrito Federal     | DF |


In [5]:
# Criamos uma coluna com o código da UF
df_suicidio['UF'] = df_suicidio['CODMUNOCOR'].apply(lambda x: str(x)[0:2])

# Criamos outra coluna com o ano da ocorrência
df_suicidio['ANOOCOR'] = df_suicidio['DTOBITO'].apply(lambda x: str(x)[-4:])

In [25]:
### Ocorrências por ano e UF
df_ano_uf = df_suicidio.groupby(['ANOOCOR', 'UF'])['DTOBITO'].count()

# Gera arquivo JSON
with open('data/ano_uf.json', 'w') as json_file:
    json_file.write( df_ano_uf.reset_index().to_json(orient='records') )
    json_file.close()

# Organiza direito o JSON
json_organizado = dict()

with open('data/ano_uf.json') as file:
    json_data = json.load(file)
    for dt in json_data:
        if dt['ANOOCOR'] in json_organizado:
            json_organizado[ dt['ANOOCOR'] ].update({ dt['UF'] : dt['DTOBITO'] })
        else:
            json_organizado.update({ dt['ANOOCOR'] : dict() })

# Gera o arquivo JSON adequado
with open('data/ano_uf.json', 'w') as file:
    json.dump(json_organizado, file)

In [7]:
### Pegar a quantidade de ocorrências por data
datas_obito = dict()

# Pego a quantidade de registros de cada data
for dtobito in df_suicidio.DTOBITO:
    if dtobito in datas_obito:
        datas_obito[ dtobito ] += 1
    else:
        datas_obito[ dtobito ] = 1
        
# Modifico as chaves do dicionario
for dtobito in df_suicidio.DTOBITO.unique():
    dtobito = str(dtobito)
    if len(dtobito) == 7:
        new_dtobito = '-'.join([dtobito[3:], dtobito[1:3], ('0'+dtobito[0])])
    else:
        new_dtobito = '-'.join([dtobito[4:], dtobito[2:4], dtobito[0:2]])
        
    datas_obito[new_dtobito] = datas_obito.pop(int(dtobito))

# Exportar para CSV
with open('data/ocor_data.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['date', 'ocor'])
    for key, value in datas_obito.items():
        writer.writerow([str(key), str(value)])

Para análisar os suicídios em feriados, escolhemos 8 feriados:

| Data (m-d) | Feriado |
| ----------:|:------- |
|    01-01   | Confraternização Universal |
|    04-21   | Tiradentes |
|    05-01   | Dia do Trabalhador |
|    09-07   | Dia da Pátria |
|    10-12   | Nossa Senhora Aparecida |
|    11-02   | Finados |
|    11-15   | Proclamação da República |
|    12-25   | Natal |

In [51]:
### Suicídios nos feriados
feriados = ['01-01', '04-21', '05-01', '09-07', '10-12', '11-02', '11-15', '12-25']
feriados